In [14]:
"""
Training Pipeline for LSTMABAR
Includes data loading, training loop, and evaluation
"""

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import librosa
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import matplotlib.pyplot as plt

from lstmabar_model import LSTMABAR, LSTMABARTrainer
from musiccaps_loader import MusicCapsLoader
from improved_text_encoders import ImprovedTextEncoder

In [2]:
class MusicCapsDataset(Dataset):
    """
    PyTorch Dataset for MusicCaps with archetype annotations
    """
    
    def __init__(
        self,
        training_data_path: str,
        sample_rate: int = 44100,
        audio_duration: float = 2.0,
        augment: bool = False
    ):
        """
        Args:
            training_data_path: Path to .npz file with training data
            sample_rate: Audio sample rate
            audio_duration: Duration to load from each audio file
            augment: Whether to apply data augmentation
        """
        self.sample_rate = sample_rate
        self.audio_duration = audio_duration
        self.augment = augment
        self.target_samples = int(sample_rate * audio_duration)
        
        # Load training data
        print(f"Loading training data from {training_data_path}")
        data = np.load(training_data_path, allow_pickle=True)
        
        self.archetype_vectors = torch.from_numpy(data['archetype_vectors']).float()
        self.descriptions = data['descriptions'].tolist()
        self.audio_paths = data['audio_paths'].tolist()
        self.archetype_order = data['archetype_order'].tolist()
        
        print(f"Loaded {len(self.descriptions)} training examples")
        
        # Filter out samples with missing audio files
        self.valid_indices = self._find_valid_samples()
        print(f"Found {len(self.valid_indices)} samples with valid audio files")
    
    def _find_valid_samples(self) -> List[int]:
        """Find indices with existing audio files"""
        valid = []
        for i, audio_path in enumerate(self.audio_paths):
            if Path(audio_path).exists():
                valid.append(i)
        return valid
    
    def __len__(self) -> int:
        return len(self.valid_indices)
    
    def __getitem__(self, idx: int) -> Dict:
        """
        Get a single training sample
        
        Returns:
            Dict with 'audio', 'description', 'archetype_weights'
        """
        # Map to valid index
        actual_idx = self.valid_indices[idx]
        
        # Load audio
        audio_path = self.audio_paths[actual_idx]
        audio, sr = librosa.load(
            audio_path,
            sr=self.sample_rate,
            duration=self.audio_duration
        )
        
        # Pad or trim to exact length
        if len(audio) < self.target_samples:
            audio = np.pad(audio, (0, self.target_samples - len(audio)))
        else:
            audio = audio[:self.target_samples]
        
        # Apply augmentation if enabled
        if self.augment:
            audio = self._augment_audio(audio)
        
        # Convert to tensor
        audio_tensor = torch.from_numpy(audio).float()
        
        # Get description and archetype weights
        description = self.descriptions[actual_idx]
        archetype_weights = self.archetype_vectors[actual_idx]
        
        return {
            'audio': audio_tensor,
            'description': description,
            'archetype_weights': archetype_weights
        }
    
    def _augment_audio(self, audio: np.ndarray) -> np.ndarray:
        """Apply random audio augmentations"""
        # Random gain (±3dB)
        if np.random.random() > 0.5:
            gain_db = np.random.uniform(-3, 3)
            audio = audio * (10 ** (gain_db / 20))
        
        # Random time shift
        if np.random.random() > 0.5:
            shift = np.random.randint(-self.sample_rate // 10, self.sample_rate // 10)
            audio = np.roll(audio, shift)
        
        # Add slight noise
        if np.random.random() > 0.5:
            noise = np.random.randn(len(audio)) * 0.005
            audio = audio + noise
        
        return audio


def collate_fn(batch: List[Dict]) -> Dict:
    """
    Custom collate function for DataLoader
    Handles variable-length descriptions
    """
    audio = torch.stack([item['audio'] for item in batch])
    descriptions = [item['description'] for item in batch]
    archetype_weights = torch.stack([item['archetype_weights'] for item in batch])
    
    return {
        'audio': audio,
        'descriptions': descriptions,
        'archetype_weights': archetype_weights
    }

In [3]:
class TrainingPipeline:
    """
    Complete training pipeline for LSTMABAR
    """
    
    def __init__(
        self,
        model: LSTMABAR,
        train_dataset: MusicCapsDataset,
        val_dataset: Optional[MusicCapsDataset] = None,
        batch_size: int = 16,
        learning_rate: float = 1e-4,
        num_epochs: int = 50,
        checkpoint_dir: str = 'checkpoints',
        log_interval: int = 10
    ):
        self.model = model
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.checkpoint_dir = Path(checkpoint_dir)
        self.log_interval = log_interval
        
        # Create checkpoint directory
        self.checkpoint_dir.mkdir(exist_ok=True)
        
        # Create data loaders
        self.train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            collate_fn=collate_fn,
            num_workers=0,
            pin_memory=True
        )
        
        if val_dataset is not None:
            self.val_loader = DataLoader(
                val_dataset,
                batch_size=batch_size,
                shuffle=False,
                collate_fn=collate_fn,
                num_workers=0,
                pin_memory=True
            )
        else:
            self.val_loader = None
        
        # Initialize trainer
        self.trainer = LSTMABARTrainer(
            model,
            learning_rate=learning_rate,
            loss_weights={
                'contrastive': 0.7,
                'archetype_prediction': 0.2,
                'audio_archetype_supervision': 0.1
            }
        )
        
        print(f"Training pipeline initialized:")
        print(f"  Training samples: {len(train_dataset)}")
        print(f"  Validation samples: {len(val_dataset) if val_dataset else 0}")
        print(f"  Batch size: {batch_size}")
        print(f"  Total epochs: {num_epochs}")
        print(f"  Steps per epoch: {len(self.train_loader)}")
    
    def train(self):
        """Run complete training loop"""
        best_val_loss = float('inf')
        
        for epoch in range(self.num_epochs):
            print(f"\n{'='*60}")
            print(f"Epoch {epoch+1}/{self.num_epochs}")
            print(f"{'='*60}")
            
            # Training
            train_losses = self.trainer.train_epoch(self.train_loader, epoch)
            print(f"\nTrain Losses: {train_losses}")
            
            # Validation
            if self.val_loader is not None:
                val_losses = {'total': 0.0}
                for batch in self.val_loader:
                    batch_losses = self.trainer.validate(
                        batch['descriptions'],
                        batch['audio'].to(self.model.device),
                        batch['archetype_weights'].to(self.model.device)
                    )
                    for key in batch_losses:
                        val_losses[key] = val_losses.get(key, 0.0) + batch_losses[key]
                
                # Average validation losses
                for key in val_losses:
                    val_losses[key] /= len(self.val_loader)
                
                print(f"Val Losses: {val_losses}")
                
                # Save best model
                if val_losses['total'] < best_val_loss:
                    best_val_loss = val_losses['total']
                    save_path = self.checkpoint_dir / 'best_model.pth'
                    self.model.save_checkpoint(
                        str(save_path),
                        epoch,
                        self.trainer.optimizer.state_dict()
                    )
                    print(f"✓ Best model saved (val_loss: {best_val_loss:.4f})")
            
            # Save periodic checkpoint
            if (epoch + 1) % 10 == 0:
                save_path = self.checkpoint_dir / f'checkpoint_epoch_{epoch+1}.pth'
                self.model.save_checkpoint(
                    str(save_path),
                    epoch,
                    self.trainer.optimizer.state_dict()
                )
        
        print(f"\n{'='*60}")
        print("Training complete!")
        print(f"{'='*60}")
        
        # Save final model
        final_path = self.checkpoint_dir / 'final_model.pth'
        self.model.save_checkpoint(
            str(final_path),
            self.num_epochs - 1,
            self.trainer.optimizer.state_dict()
        )
        
        # Plot training history
        self.plot_training_history()
    
    def plot_training_history(self):
        """Plot training curves"""
        history = self.trainer.history
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Total loss
        axes[0].plot(history['train_loss'], label='Train Loss')
        if history['val_loss']:
            axes[0].plot(history['val_loss'], label='Val Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Total Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Component losses
        axes[1].plot(history['contrastive_loss'], label='Contrastive Loss')
        axes[1].plot(history['archetype_loss'], label='Archetype Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].set_title('Component Losses')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.checkpoint_dir / 'training_history.png', dpi=300)
        print(f"Training history plot saved to {self.checkpoint_dir / 'training_history.png'}")
        plt.close()


def prepare_musiccaps_data(
    csv_path: str,
    audio_dir: str = 'musiccaps_audio',
    output_path: str = 'musiccaps_training_data.npz',
    max_downloads: int = 5000,
    train_split: float = 0.7,
    val_split: float = 0.15,
    test_split: float = 0.15,
    random_seed: int = 42
) -> Tuple[str, str]:
    """
    Prepare MusicCaps dataset for training

    Args:
        csv_path: Path to MusicCaps CSV
        audio_dir: Directory to save audio
        output_path: Base path for output files
        max_downloads: Max clips to download
        train_split: Fraction for training (default 0.7)
        val_split: Fraction for validation (default 0.15)
        test_split: Fraction for testing (default 0.15)
        random_seed: Random seed for reproducibility
    
    Returns:
        Tuple of (train_path, val_path, test_path) .npz files
    """

    assert abs(train_split + val_split + test_split - 1.0) < 1e-6, \
        "Splits must sum to 1.0"

    print("=== Preparing MusicCaps Dataset ===")
    
    # Load and process MusicCaps
    loader = MusicCapsLoader(csv_path, audio_dir)
    
    # Download audio clips
    print(f"\nDownloading up to {max_downloads} audio clips...")
    downloaded, failed = loader.download_audio_clips(
        max_clips=max_downloads,
        use_balanced_subset=True
    )
    
    print(f"Successfully downloaded: {len(downloaded)}")
    print(f"Failed: {len(failed)}")
    
    # Create archetype training data
    print("\nCreating archetype training data...")
    training_data = loader.create_archetype_training_data(use_tfidf_weighting=True)
    
    # Shuffle data for random splits
    np.random.seed(random_seed)
    indices = np.random.permutation(len(training_data))
    training_data = [training_data[i] for i in indices]
    
    # Calculate split points
    n_total = len(training_data)
    n_train = int(n_total * train_split)
    n_val = int(n_total * val_split)
    n_test = n_total - n_train - n_val
    
    # Split data
    train_data = training_data[:n_train]
    val_data = training_data[n_train:n_train + n_val]
    test_data = training_data[n_train + n_val:]
    
    print(f"\nDataset split:")
    print(f"  Train: {len(train_data)} samples ({train_split*100:.1f}%)")
    print(f"  Val:   {len(val_data)} samples ({val_split*100:.1f}%)")
    print(f"  Test:  {len(test_data)} samples ({test_split*100:.1f}%)")
    
    # Save train set
    train_path = output_path.replace('.npz', '_train.npz')
    vectors_train = np.array([item['archetype_vector'] for item in train_data])
    descriptions_train = [item['description'] for item in train_data]
    audio_paths_train = [item['audio_path'] for item in train_data]
    
    np.savez_compressed(
        train_path,
        archetype_vectors=vectors_train,
        descriptions=descriptions_train,
        audio_paths=audio_paths_train,
        archetype_order=train_data[0]['archetype_order']
    )
    print(f"\n✓ Saved train set: {train_path}")
    
    # Save val set
    val_path = output_path.replace('.npz', '_val.npz')
    vectors_val = np.array([item['archetype_vector'] for item in val_data])
    descriptions_val = [item['description'] for item in val_data]
    audio_paths_val = [item['audio_path'] for item in val_data]
    
    np.savez_compressed(
        val_path,
        archetype_vectors=vectors_val,
        descriptions=descriptions_val,
        audio_paths=audio_paths_val,
        archetype_order=val_data[0]['archetype_order']
    )
    print(f"✓ Saved val set: {val_path}")
    
    # Save test set
    test_path = output_path.replace('.npz', '_test.npz')
    vectors_test = np.array([item['archetype_vector'] for item in test_data])
    descriptions_test = [item['description'] for item in test_data]
    audio_paths_test = [item['audio_path'] for item in test_data]
    
    np.savez_compressed(
        test_path,
        archetype_vectors=vectors_test,
        descriptions=descriptions_test,
        audio_paths=audio_paths_test,
        archetype_order=test_data[0]['archetype_order']
    )
    print(f"✓ Saved test set: {test_path}")
    
    return train_path, val_path, test_path

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}\n")

# # Old Baseline Configuration
# config = {
#     'embedding_dim': 768,
#     'audio_architecture': 'resnet',  # or 'ast'
#     'sample_rate': 44100,
#     'audio_duration': 2.0,
#     'batch_size': 16,
#     'learning_rate': 1e-4,
#     'num_epochs': 20,
#     'max_downloads': 5000
# }

# Approach C Improved Config
# New Configuration
config = {
    'embedding_dim': 768,
    'audio_architecture': 'resnet',
    'audio_duration': 2.0,
    'sample_rate': 44100,
    'batch_size': 32,  # Increased from 16
    'learning_rate': 3e-4,  # Increased from 1e-4
    'num_epochs': 30,  # Increased from 20
    'weight_decay': 0.01,  # Added regularization
    'max_downloads': 5000
}

Using device: cpu



In [5]:
# Step 0: Download data (run once)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("googleai/musiccaps")

print("Path to dataset files:", path)

csv_path = f"{path}/musiccaps-public.csv"

Path to dataset files: /home/zain/.cache/kagglehub/datasets/googleai/musiccaps/versions/1


In [6]:
# Step 1: Prepare dataset (run once)
print("Step 1: Preparing MusicCaps dataset with train/val/test split...")
train_data_path, val_data_path, test_data_path = prepare_musiccaps_data(
    csv_path=csv_path,
    audio_dir='musiccaps_audio',
    max_downloads=config['max_downloads'],
    train_split=0.7,
    val_split=0.15,
    test_split=0.15,
    random_seed=42
)

Step 1: Preparing MusicCaps dataset with train/val/test split...
=== Preparing MusicCaps Dataset ===
Loaded 5521 MusicCaps examples

Using balanced subset: 1000 examples
✓ Already exists: -bgHkxwoliw
✓ Already exists: -kpR93atgd8
✓ Already exists: -wymN80CiYU
✓ Already exists: 07xGXxIHOL4
✓ Already exists: 0PMFAO4TIU4
✓ Already exists: 0TV9zvfwFhs
✓ Already exists: 0fiOM---7QI


ERROR: [youtube] 0i8VM_EooCs: Video unavailable. This video is no longer available due to a copyright claim by Terrabyte Music Limited


Error downloading 0i8VM_EooCs: ERROR: [youtube] 0i8VM_EooCs: Video unavailable. This video is no longer available due to a copyright claim by Terrabyte Music Limited
✗ Failed: 0i8VM_EooCs
✓ Already exists: 0jFQ21A6GRA
✓ Already exists: 1ACn3u5UnBw
✓ Already exists: 1BVSYfNCcv0
✓ Already exists: 1JpeDWbgUO8
✓ Already exists: 1PKxdTlquCA
✓ Already exists: 1Q9DXhXMSFI
✓ Already exists: 1TyOPtg0Yfk
✓ Already exists: 1V7ReAk9k-4
✓ Already exists: 1j4rFfU5XKQ
✓ Already exists: 20Vh6z6Ie0E


ERROR: [youtube] 2G5bSYHcJSM: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Error downloading 2G5bSYHcJSM: ERROR: [youtube] 2G5bSYHcJSM: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
✗ Failed: 2G5bSYHcJSM
✓ Already exists: 2GWkKVHxGRM
✓ Already exists: 2JnlmS1zzls
✓ Already exists: 2RU4CSDzS-g
✓ Already exists: 2U8Dvh7nwFI
✓ Already exists: 2ZfthfWQowE
✓ Already exists: 2bCuw7U_Rac
✓ Already exists: 2dyEnOo3yJ8
✓ Already exists: 2vQTq4QLP8U
✓ Already exists: 2xGRCsW6-Bk
✓ Already exists: 3JYQgXudiH8
✓ Already exists: 3TQmts_MxyQ
✓ Already exists: 40D4L5Ndi6k
✓ Already exists: 44sbWBFswUY
✓ Already exists: 4i11P4OCRfk
✓ Already exists: 5JQIsqc8HBc
✓ Already exists: 5XXAeSybGK0
✓ Already exists: 5ZpVhmhVYoI
✓ Already exists: 5_orEetudIA
✓ Already exists: 5gyMt0YzPQ0
✓ Already exists: 60OIHit4Q-M
✓ Already exists: 6N1LWG4aztA
✓ Already exists: 6k4lcF9IGUk
✓ Already exists: 7-mNJ4IUY5Q
✓ Already exists: 7WZwlOrRELI
✓ Already exists: 7_q36NyJtQY
✓ Already exists: 8BJljuSm2Aw
✓ Alread

...

Error downloading yzT9nsHslAk: ERROR: [youtube] yzT9nsHslAk: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
✗ Failed: yzT9nsHslAk
✓ Already exists: z0w3Y8BGLQE
✓ Already exists: z2kTJ6pQ4Uo


ERROR: [youtube] z4ewUsnIJKI: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error downloading z4ewUsnIJKI: ERROR: [youtube] z4ewUsnIJKI: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
✗ Failed: z4ewUsnIJKI
✓ Already exists: z6SNngkMAug
✓ Already exists: z7vNtEcM9Bk
✓ Already exists: z8Wjdss5uMg
✓ Already exists: z9hRQiJMnIw


ERROR: [youtube] zCrpaLEq1VQ: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error downloading zCrpaLEq1VQ: ERROR: [youtube] zCrpaLEq1VQ: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
✗ Failed: zCrpaLEq1VQ
✓ Already exists: zD4PXuUkwsc
✓ Already exists: zGBKakEGSyc
✓ Already exists: zGONTDY_rNM
✓ Already exists: zNSV9GkpRfM
✓ Already exists: zNbF006Y5x4
✓ Already exists: zNgRUFOj3HI
✓ Already exists: zOvFaef41iw
✓ Already exists: zPMiMXCazAI
✓ Already exists: zPhuyMYy9EI
✓ Already exists: zR4ebAuqy8w
✓ Already exists: zRixZ7mWwho
✓ Already exists: zU4mwg-HHoQ
✓ Already exists: zUaZGKZIKic
✓ Already exists: zWJC_qr2610
✓ Already exists: zXxJymYt8Z4
✓ Already exists: zYM0gtd_PRo
✓ Already exists: zYUZEXCE7gw
✓ Already exists: z_aUtjx

In [20]:
# Step 2: Create datasets
print("\nStep 2: Creating PyTorch datasets...")
train_dataset = MusicCapsDataset(
    train_data_path,
    sample_rate=config['sample_rate'],
    audio_duration=config['audio_duration'],
    augment=True
)

val_dataset = MusicCapsDataset(
    val_data_path,
    sample_rate=config['sample_rate'],
    audio_duration=config['audio_duration'],
    augment=False
)

test_dataset = MusicCapsDataset(
    test_data_path,
    sample_rate=config['sample_rate'],
    audio_duration=config['audio_duration'],
    augment=False
)


Step 2: Creating PyTorch datasets...
Loading training data from musiccaps_training_data_train.npz
Loaded 669 training examples
Found 669 samples with valid audio files
Loading training data from musiccaps_training_data_val.npz
Loaded 143 training examples
Found 143 samples with valid audio files
Loading training data from musiccaps_training_data_test.npz
Loaded 145 training examples
Found 145 samples with valid audio files


In [21]:
# Cell 1: Reload the modules to pick up changes
import importlib
import text_tower
import lstmabar_model

importlib.reload(text_tower)
importlib.reload(lstmabar_model)

# Re-import the classes
from text_tower import TextEncoder
from lstmabar_model import LSTMABAR, LSTMABARTrainer
from improved_text_encoders import ImprovedTextEncoder

In [22]:
# Step 3: Initialize model
print("\nStep 3: Initializing LSTMABAR model...")
model = LSTMABAR(
    embedding_dim=config['embedding_dim'],
    audio_architecture=config['audio_architecture'],
    sample_rate=config['sample_rate'],
    use_quantum_attention=False,
    device=device
)

# Approach C Better base model
model.text_encoder = ImprovedTextEncoder(
    model_name='sentence-transformers/all-mpnet-base-v2',
    embedding_dim=768,
    projection_depth='deep',
    device=device
)

print(f"Model has {sum(p.numel() for p in model.parameters()):,} parameters")


Step 3: Initializing LSTMABAR model...
Loading text encoder: sentence-transformers/all-MiniLM-L6-v2
Loading improved text encoder: sentence-transformers/all-mpnet-base-v2
Model has 128,403,725 parameters


# Approach C. Do this or A or B, not all 3

In [24]:
# Step 4C: Configure freezing (Approach A style)

# Freeze everything except projection and contrastive
for p in model.parameters():
    p.requires_grad = False

# Unfreeze projection
for p in model.text_encoder.projection.parameters():
    p.requires_grad = True

# Unfreeze contrastive module
if hasattr(model, 'contrastive_module'):
    for p in model.contrastive_module.parameters():
        p.requires_grad = True

print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Trainable params: 6,233,091


In [25]:
# Step 5C: Train!

# Set checkpoint directory
checkpoint_dir = Path("checkpoints/improved_approach_c")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# Create pipeline
pipeline = TrainingPipeline(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=config['batch_size'],
    learning_rate=config['learning_rate'],
    num_epochs=config['num_epochs'],
    checkpoint_dir=checkpoint_dir
)

# Train
pipeline.train()

Training pipeline initialized:
  Training samples: 669
  Validation samples: 143
  Batch size: 32
  Total epochs: 30
  Steps per epoch: 21

Epoch 1/30
Epoch 0, Batch 0/21: Loss=2.4596, Contrastive=3.4719, Archetype=0.1002
Epoch 0, Batch 10/21: Loss=2.4557, Contrastive=3.4575, Archetype=0.1232
Epoch 0, Batch 20/21: Loss=2.3849, Contrastive=3.3693, Archetype=0.0926

Train Losses: {'total': np.float64(2.4606422129131498), 'contrastive': np.float64(3.47143208412897), 'archetype': np.float64(0.10547633540062677)}
Val Losses: {'total': 2.351073884963989, 'contrastive': 3.3214312553405763, 'archetype': 0.08572169691324234}
Checkpoint saved to checkpoints/improved_approach_c/best_model.pth
✓ Best model saved (val_loss: 2.3511)

Epoch 2/30
Epoch 1, Batch 0/21: Loss=2.4647, Contrastive=3.4699, Archetype=0.1237
Epoch 1, Batch 10/21: Loss=2.4388, Contrastive=3.4383, Archetype=0.1121
Epoch 1, Batch 20/21: Loss=2.3687, Contrastive=3.3383, Archetype=0.1120

Train Losses: {'total': np.float64(2.421911

## Do not do both A approach and B approach, just choose one or the other

### Fine Tuning approach A: Use Step 4A, Step 4 and then 5A. DO NOT RUN BOTH 5 AND 5A!

In [ ]:
# Step 4A: Fine-tuning  Approach A (Projection-only)

# 1) Freeze everything by default
for p in model.parameters():
    p.requires_grad = False

# 2) Unfreeze ONLY the text projection head
assert hasattr(model, "text_encoder") and hasattr(model.text_encoder, "projection"), \
    "Expected model.text_encoder.projection to exist."
for p in model.text_encoder.projection.parameters():
    p.requires_grad = True

# 3) Keep contrastive module trainable (e.g., temperature)
if hasattr(model, "contrastive_module"):
    for p in model.contrastive_module.parameters():
        p.requires_grad = True

# 4) Sanity check: count trainable params
def _count_trainable(m): 
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters (total): {total_trainable:,}")
print(f" - text projection: {_count_trainable(model.text_encoder.projection):,}")
if hasattr(model, "contrastive_module"):
    print(f" - contrastive module: {_count_trainable(model.contrastive_module):,}")

# 5) Use a slightly higher LR for this phase (TrainingPipeline passes this to LSTMABARTrainer)
config['learning_rate'] = 5e-4

Trainable parameters (total): 1,806,338
 - text projection: 887,808
 - contrastive module: 918,530


### Fine Tuning approach B: Use Step 4B, Step 4 and then 5B. DO NOT RUN BOTH 5 AND 5B!

In [11]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

class HFBackboneTextEncoder(nn.Module):
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2',
                 embedding_dim=768, device='cpu'):
        super().__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.backbone = AutoModel.from_pretrained(model_name)  # MiniLM
        self.base_dim = self.backbone.config.hidden_size

        self.projection = nn.Sequential(
            nn.Linear(self.base_dim, embedding_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embedding_dim, embedding_dim),
            nn.LayerNorm(embedding_dim),
        )
        self.to(device)

    def _mean_pool(self, token_embeddings, attention_mask):
        mask = attention_mask.unsqueeze(-1).float()
        summed = (token_embeddings * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1e-6)
        return summed / denom

    def forward(self, texts):
        tokens = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
        outputs = self.backbone(**tokens)                     # [B, T, H]
        pooled = self._mean_pool(outputs.last_hidden_state, tokens.attention_mask)  # [B, H]
        emb = self.projection(pooled)                         # [B, D]
        return F.normalize(emb, p=2, dim=1)

# 1) Swap the model's text encoder with the HF-backbone version
model.text_encoder = HFBackboneTextEncoder(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    embedding_dim=config['embedding_dim'],
    device=device
)

# 2) Freeze everything first
for p in model.parameters():
    p.requires_grad = False

# 3) Unfreeze ONLY the last 2 transformer blocks + the projection head
# MiniLM-L6 has encoder.layer.0 ... encoder.layer.5; we unfreeze 4 and 5
for name, p in model.text_encoder.backbone.named_parameters():
    if name.startswith("encoder.layer.4") or name.startswith("encoder.layer.5"):
        p.requires_grad = True

for p in model.text_encoder.projection.parameters():
    p.requires_grad = True

# Keep contrastive temperature learnable
if hasattr(model, "contrastive_module"):
    for p in model.contrastive_module.parameters():
        p.requires_grad = True

# 4) Sanity: print trainable counts
def _count_trainable(m): 
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters (total): {total_trainable:,}")
print(f" - text projection: {_count_trainable(model.text_encoder.projection):,}")

# Count unfrozen backbone params
bb_trainable = 0
for n, p in model.text_encoder.backbone.named_parameters():
    if p.requires_grad: bb_trainable += p.numel()
print(f" - text backbone (last 2 layers): {bb_trainable:,}")

# 5) Use a conservative LR for backbone tuning (single LR, trainer builds the optimizer)
config['learning_rate'] = 1e-4

# 6) Write to a separate checkpoint folder to avoid overwriting
from pathlib import Path
ckpt_dir_B = Path("checkpoints/fine_tune_B")
ckpt_dir_B.mkdir(parents=True, exist_ok=True)

Trainable parameters (total): 5,355,266
 - text projection: 887,808
 - text backbone (last 2 layers): 3,548,928


In [12]:
# Step 4: Create training pipeline
print("\nStep 4: Setting up training pipeline...")
pipeline = TrainingPipeline(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=config['batch_size'],
    learning_rate=config['learning_rate'],
    num_epochs=config['num_epochs'],
    checkpoint_dir='checkpoints'
)


Step 4: Setting up training pipeline...
Training pipeline initialized:
  Training samples: 669
  Validation samples: 143
  Batch size: 16
  Total epochs: 20
  Steps per epoch: 42


In [ ]:
# Step 5A: Train!Fine-tuning  Approach A (Projection-only) ONLY

print("\nStep 5A: Starting training...")
pipeline.checkpoint_dir = Path("checkpoints/fine_tune_A")
pipeline.checkpoint_dir.mkdir(exist_ok=True, parents=True)
pipeline.train()

print("\n✓ Training complete! Model checkpoints saved to checkpoints/fine_tune_A")


Step 5A: Starting training...

Epoch 1/20
Epoch 0, Batch 0/21: Loss=1.9739, Contrastive=2.7776, Archetype=0.0990
Epoch 0, Batch 10/21: Loss=1.9679, Contrastive=2.7758, Archetype=0.0866
Epoch 0, Batch 20/21: Loss=1.8013, Contrastive=2.5328, Archetype=0.0963

Train Losses: {'total': np.float64(1.9669640064239502), 'contrastive': np.float64(2.7668586117880687), 'archetype': np.float64(0.10301079423654647)}
Val Losses: {'total': 1.8935595989227294, 'contrastive': 2.664562702178955, 'archetype': 0.09419761896133423}
Checkpoint saved to checkpoints/fine_tune_A/best_model.pth
✓ Best model saved (val_loss: 1.8936)

Epoch 2/20
Epoch 1, Batch 0/21: Loss=1.9037, Contrastive=2.6735, Archetype=0.1133
Epoch 1, Batch 10/21: Loss=1.9206, Contrastive=2.6979, Archetype=0.1123
Epoch 1, Batch 20/21: Loss=1.7890, Contrastive=2.5132, Archetype=0.1014

Train Losses: {'total': np.float64(1.909187952677409), 'contrastive': np.float64(2.684551965622675), 'archetype': np.float64(0.10219607182911464)}
Val Losses

In [13]:
# Step 5B: Train!Fine-tuning  Approach A (Projection-only) ONLY

print("\nStep 5B: Starting training...")
pipeline.checkpoint_dir = Path("checkpoints/fine_tune_B")
pipeline.checkpoint_dir.mkdir(exist_ok=True, parents=True)
pipeline.train()

print("\n✓ Training complete! Model checkpoints saved to checkpoints/fine_tune_B")


Step 5B: Starting training...

Epoch 1/20


/home/zain/anaconda3/envs/lab3py313/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 0, Batch 0/42: Loss=1.9790, Contrastive=2.7697, Archetype=0.1402
Epoch 0, Batch 10/42: Loss=1.9659, Contrastive=2.7584, Archetype=0.1212
Epoch 0, Batch 20/42: Loss=1.9612, Contrastive=2.7516, Archetype=0.1155
Epoch 0, Batch 30/42: Loss=1.9381, Contrastive=2.7245, Archetype=0.1051
Epoch 0, Batch 40/42: Loss=1.8933, Contrastive=2.6678, Archetype=0.0879

Train Losses: {'total': np.float64(1.966353450502668), 'contrastive': np.float64(2.7659767866134644), 'archetype': np.float64(0.10304137957947594)}


/home/zain/anaconda3/envs/lab3py313/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Val Losses: {'total': 1.9410083956188626, 'contrastive': 2.733148733774821, 'archetype': 0.09454374180899726}
Checkpoint saved to checkpoints/fine_tune_B/best_model.pth
✓ Best model saved (val_loss: 1.9410)

Epoch 2/20
Epoch 1, Batch 0/42: Loss=1.9640, Contrastive=2.7560, Archetype=0.1205
Epoch 1, Batch 10/42: Loss=1.9676, Contrastive=2.7663, Archetype=0.1109
Epoch 1, Batch 20/42: Loss=1.9164, Contrastive=2.6975, Archetype=0.0910
Epoch 1, Batch 30/42: Loss=1.9224, Contrastive=2.7106, Archetype=0.0855
Epoch 1, Batch 40/42: Loss=1.7932, Contrastive=2.5238, Archetype=0.0928

Train Losses: {'total': np.float64(1.906069034621829), 'contrastive': np.float64(2.679955925260271), 'archetype': np.float64(0.1027061307714099)}
Val Losses: {'total': 1.9074376026789348, 'contrastive': 2.687011374367608, 'archetype': 0.0881727925605244}
Checkpoint saved to checkpoints/fine_tune_B/best_model.pth
✓ Best model saved (val_loss: 1.9074)

Epoch 3/20
Epoch 2, Batch 0/42: Loss=1.9083, Contrastive=2.6791, Arc

In [ ]:
# Step 5: Train!
print("\nStep 5: Starting training...")
pipeline.train()

print("\n✓ Training complete! Model checkpoints saved to checkpoints/")


Step 5: Starting training...

Epoch 1/20


/home/zain/anaconda3/envs/lab3py313/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 0, Batch 0/21: Loss=1.9772, Contrastive=2.7725, Archetype=0.1295
Epoch 0, Batch 10/21: Loss=1.9883, Contrastive=2.7958, Archetype=0.1115
Epoch 0, Batch 20/21: Loss=1.7641, Contrastive=2.4914, Archetype=0.0669

Train Losses: {'total': np.float64(1.9617775565101987), 'contrastive': np.float64(2.7585508823394775), 'archetype': np.float64(0.10732915713673546)}


/home/zain/anaconda3/envs/lab3py313/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Val Losses: {'total': 1.8791366338729858, 'contrastive': 2.6434515714645386, 'archetype': 0.09720117449760438}
Checkpoint saved to checkpoints/best_model.pth
✓ Best model saved (val_loss: 1.8791)

Epoch 2/20
Epoch 1, Batch 0/21: Loss=1.8743, Contrastive=2.6430, Archetype=0.0831
Epoch 1, Batch 10/21: Loss=1.8963, Contrastive=2.6557, Archetype=0.1267
Epoch 1, Batch 20/21: Loss=1.5718, Contrastive=2.2092, Archetype=0.0885

Train Losses: {'total': np.float64(1.8077486412865775), 'contrastive': np.float64(2.5401792299179804), 'archetype': np.float64(0.10258456213133675)}
Val Losses: {'total': 1.8059094190597533, 'contrastive': 2.5391037940979, 'archetype': 0.09716752469539643}
Checkpoint saved to checkpoints/best_model.pth
✓ Best model saved (val_loss: 1.8059)

Epoch 3/20
Epoch 2, Batch 0/21: Loss=1.6950, Contrastive=2.3698, Archetype=0.1257
Epoch 2, Batch 10/21: Loss=1.6389, Contrastive=2.3039, Archetype=0.0885
Epoch 2, Batch 20/21: Loss=1.3446, Contrastive=1.8780, Archetype=0.1042

Train 